# 1. Introduction:

Connect Four is a classic two-player board game where the objective is to be the first player to create a line of four of your own tokens either horizontally, vertically, or diagonally on a grid of 7 columns and 6 rows. Players take turns dropping tokens from the top into any of the columns, with the token occupying the lowest available space in the chosen column.

Objective of the Solution:

The objective of our solution is to implement two different algorithms, namely A* search and Monte Carlo Tree Search (MCTS), to create an AI player capable of playing Connect Four against a human player. The AI player should be able to strategically evaluate the game state and make intelligent moves to either win the game or prevent the opponent from winning. By implementing these algorithms, we aim to provide a challenging opponent for human players and explore different approaches to solving the game. We will compare the performance of these algorithms in terms of their ability to make optimal moves and provide insights into their strengths and weaknesses.

# 2. Algorithm Descriptions:

## 2.1. A* Search Algorithm:
   - A* (pronounced as "A-star") is a widely-used graph traversal and pathfinding algorithm that finds the shortest path between a given start node and a target node in a weighted graph.
   - It works by maintaining a priority queue of nodes to be evaluated, prioritizing nodes based on a combination of the cost of reaching that node from the start node (known as the "g-value") and a heuristic estimate of the cost from that node to the target node (known as the "h-value").
   - At each step, A* selects the node with the lowest combined cost (f-value) and expands it by considering its neighboring nodes. It then updates the cost and heuristic values for these neighbors and adds them to the priority queue.
   - The process continues until the target node is reached or the priority queue becomes empty.

### 2.1.1 Implementation in Code:
   - In the provided code, the A* search algorithm is adapted to find the best move for the AI player in Connect Four.
   - The algorithm evaluates each possible move by dropping a token in each column and calculates a score for each resulting game state using a heuristic evaluation function.
   - The move with the highest score is selected as the best move for the AI player.

## 2.2 Monte Carlo Tree Search (MCTS) Algorithm:
   - Monte Carlo Tree Search (MCTS) is a probabilistic search algorithm used for decision-making in games and other domains with large search spaces and uncertain outcomes.
   - It operates by iteratively building a search tree where each node represents a possible state of the game, and the edges represent possible actions.
   - MCTS consists of four main steps: selection, expansion, simulation (also known as playout or rollout), and backpropagation.
   - During the selection phase, the algorithm navigates the tree by selecting nodes according to a selection policy (e.g., Upper Confidence Bound).
   - In the expansion phase, the algorithm expands the selected node by adding child nodes representing possible actions.
   - The simulation phase involves randomly playing out the game from the selected node until a terminal state is reached.
   - Finally, the algorithm updates the node statistics (e.g., visits and wins) and propagates the result back up the tree during the backpropagation phase.

### 2.2.1 How it's Used for Decision-Making in Connect Four:
   - In Connect Four, MCTS is used to explore the game tree and find the best move for the AI player.
   - The algorithm simulates multiple games starting from the current game state by randomly choosing moves until a terminal state (win, loss, or draw) is reached.
   - After a specified number of simulations, the algorithm selects the move that leads to the highest win rate among the simulated games.
   - By iteratively improving the estimate of move quality based on simulation results, MCTS can effectively search for optimal moves in Connect Four.

# 3. Code Implementation

The provided code implements the classic game of Connect Four along with two different algorithms for making optimal moves: A* search and Monte Carlo Tree Search (MCTS). Connect Four is a two-player game where the objective is to be the first to form a horizontal, vertical, or diagonal line of four tokens of your own color on a grid of 6 rows and 7 columns.

In this implementation, the game logic is encapsulated within the ConnectFour class, which handles the game board, player turns, token dropping, and win/draw conditions. Additionally, there is a set of functions for evaluating the game board and finding the best move for the AI player.

The A* search algorithm is utilized to perform an exhaustive search of the game tree, evaluating potential moves using a heuristic evaluation function. It aims to find the move that maximizes the AI player's chances of winning while minimizing the opponent's chances.

On the other hand, the Monte Carlo Tree Search (MCTS) algorithm employs a probabilistic approach to decision-making. It simulates multiple games from the current game state, randomly selecting moves until a terminal state is reached. By iteratively updating statistics and exploring promising branches of the game tree, MCTS identifies the optimal move for the AI player.

Throughout the code, detailed comments are provided to explain the functionality and logic of each component, ensuring clarity and ease of understanding. With these algorithms, the Connect Four game becomes an engaging platform for exploring different strategies and honing decision-making skills.

## 3.1 Imports and Constants

1. `WIN_X`: This constant represents the score assigned when player X wins the game. In Connect Four, winning the game results in a high positive score to indicate a favorable outcome for player X.

2. `WIN_O`: Similar to `WIN_X`, this constant represents the score assigned when player O wins the game. However, it's assigned a negative value to indicate a favorable outcome for player O.

3. `DRAW`: This constant represents the score assigned when the game ends in a draw. In Connect Four, a draw occurs when the board is full and neither player has achieved victory. It's typically assigned a neutral or zero score.

4. `MOVE_BONUS_X`: This constant represents the bonus score given to player X for making a move. It incentivizes making moves and is typically a positive value.

5. `MOVE_BONUS_O`: Similar to `MOVE_BONUS_X`, this constant represents the bonus score given to player O for making a move. However, it's assigned a negative value to reflect the opposing nature of player O's objectives.

6. `SEGMENT_VALUES`: This constant is a dictionary that assigns scores to different combinations of tokens in a segment of the game board. The keys represent the number of tokens of a particular player in a segment, and the values represent the corresponding score for that segment configuration. Positive scores are typically assigned to player X, while negative scores are assigned to player O. This dictionary helps in evaluating the board state and determining the desirability of different configurations of tokens for each player.

In [29]:
import random
import math

WIN_X = 512
WIN_O = -512
DRAW = 0
MOVE_BONUS_X = 16
MOVE_BONUS_O = -16
SEGMENT_VALUES = {-3: -50, -2: -10, -1: -1, 0: 0, 1: 1, 2: 10, 3: 50}

## 3.2 ConnectFour

In this section we will take a closer look at all the methods of the ConnectFour class.

### 3.2.1 The `__init__` method
This line initializes the game board for Connect Four. It creates a 6x7 grid represented as a list of lists, where each element is initialized to `'.'`, indicating an empty space on the board. The outer list represents the rows of the board, and the inner lists represent the columns. 

```python
[['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.']]
```

The board layout is as follows:
- Rows are numbered from top to bottom (0 to 5).
- Columns are numbered from left to right (0 to 6).

This initialization ensures that the game board starts empty, with no tokens placed in any column.

The `current_player` attribute is set to `'X'`, indicating that the game starts with player X's turn. This attribute keeps track of the current player's turn during the game.

### 3.2.2 The `display_board` method

This method, `display_board`, is responsible for printing the current state of the game board to the console. Let's break down its functionality:

- `for row in self.board:`: This line iterates over each row in the `board` attribute of the `ConnectFour` class instance. The `board` attribute represents the 6x7 grid where tokens are placed during the game.

- `print(' '.join(row))`: Within the loop, this line joins the tokens in the current `row` with a space in between each token. It converts the row, which is initially a list of individual tokens (represented as strings), into a single string where tokens are separated by spaces. This string represents the visual representation of the row as it would appear on the game board.

- `print()`: After printing each row, this line prints an empty line. This creates a visual separation between rows, making the output clearer and more readable.

Overall, this method effectively prints the entire game board by displaying each row with tokens separated by spaces and adding empty lines between rows for clarity.

### 3.2.3 The `drop_token` method

The `drop_token` method in the `ConnectFour` class is responsible for placing a token (belonging to the current player) into a specified column of the game board. Here's an explanation of how it works:

- `def drop_token(self, col):`: This line defines the method, indicating that it belongs to the `ConnectFour` class and takes a single argument `col`, which represents the column where the token will be dropped.

- `for row in range(5, -1, -1):`: This line iterates over the rows of the specified column in reverse order, starting from the bottom row (index 5) and moving upwards to the top row (index 0). This is because tokens are dropped from the top and fall to the bottom of the column.

- `if self.board[row][col] == '.':`: Within the loop, this line checks if the current cell in the column is empty (represented by the `'.'` character). If the cell is empty, it means that the current row is a valid position to place the token.

- `self.board[row][col] = self.current_player`: If the cell is empty, this line assigns the current player's token (`'X'` or `'O'`) to the cell in the game board, indicating that the player has placed their token in that position.

- `return True`: After successfully dropping the token, this line returns `True` to indicate that the operation was successful.

- `return False`: If the loop completes without finding an empty cell in the specified column, this line is reached, indicating that the column is full and the token cannot be dropped. In this case, the method returns `False` to signify that the operation failed.

Overall, the `drop_token` method allows players to drop their tokens into specified columns of the game board, ensuring that tokens are placed in the lowest available position within the column.

### 3.2.4 The `switch_player` method

The `switch_player` method in the `ConnectFour` class is responsible for changing the current player after a turn is completed in the game. Here's an explanation of how it works:

- `def switch_player(self):`: This line defines the method, indicating that it belongs to the `ConnectFour` class and does not take any arguments.

- `self.current_player = 'X' if self.current_player == 'O' else 'O'`: This line toggles the current player between `'X'` and `'O'`. It checks the value of `self.current_player`, which stores the symbol of the current player. If the current player is `'X'`, it is changed to `'O'`, and vice versa. This conditional expression effectively switches the current player to the other player.

By calling this method after each turn in the game, the game logic ensures that players alternate turns, allowing them to take actions (such as dropping tokens) on the game board in sequence.

### 3.2.5 The `check_winner` method 

The `check_winner` method in the `ConnectFour` class is responsible for determining if there is a winner in the Connect Four game after each move. Here's an explanation of how it works:

- `def check_winner(self):`: This line defines the method, indicating that it belongs to the `ConnectFour` class and does not take any arguments.

- `for row in range(6):`: This line iterates over each row index in the game board.

- `for col in range(7):`: This line iterates over each column index in the game board.

- `if self.board[row][col] != '.':`: This line checks if the current cell in the board is occupied by a player's token (either `'X'` or `'O'`). If the cell is empty (represented by `'.'`), the loop continues to the next iteration.

- `if (self.check_line(row, col, 1, 0) or`: This line checks if there is a winning sequence horizontally by calling the `check_line` method with parameters indicating the direction of the check (1 for horizontal).

- `self.check_line(row, col, 0, 1) or`: This line checks if there is a winning sequence vertically by calling the `check_line` method with parameters indicating the direction of the check (1 for vertical).

- `self.check_line(row, col, 1, 1) or`: This line checks if there is a winning sequence diagonally (from top-left to bottom-right) by calling the `check_line` method with parameters indicating the direction of the check (1 for diagonal).

- `self.check_line(row, col, 1, -1)):`: This line checks if there is a winning sequence diagonally (from top-right to bottom-left) by calling the `check_line` method with parameters indicating the direction of the check (1 for diagonal).

- `if all(cell != '.' for row in self.board for cell in row):`: This line checks if the game board is fully occupied by player tokens, indicating a draw if there is no winner.

- `return self.board[row][col]`: If any of the checks for winning sequences is successful, the method returns the symbol of the winning player (`'X'` or `'O'`). If there is no winner, it returns `'Draw'` if the game ends in a draw.

The `check_winner` method provides a comprehensive check to determine if there is a winner in the game after each move, considering all possible winning sequences horizontally, vertically, and diagonally.

### 3.2.6 The `check_line` method 

The `check_line` method in the `ConnectFour` class is a helper method used by the `check_winner` method to determine if there is a winning sequence of tokens in a specific direction (horizontal, vertical, or diagonal) starting from a given cell position on the game board. Here's a breakdown of how it works:

- `def check_line(self, row, col, dr, dc):`: This line defines the method, indicating that it belongs to the `ConnectFour` class and takes four parameters:
  - `row`: The row index of the starting cell.
  - `col`: The column index of the starting cell.
  - `dr`: The change in row index for each step in the specified direction (`1`, `0`, or `-1`).
  - `dc`: The change in column index for each step in the specified direction (`1`, `0`, or `-1`).

- `token = self.board[row][col]`: This line retrieves the token (either `'X'` or `'O'`) from the starting cell position.

- `for _ in range(3):`: This line iterates three times to check the next three cells in the specified direction, forming a sequence of four cells in total (including the starting cell).

- `row += dr`: This line updates the row index based on the specified change in the row direction (`dr`).

- `col += dc`: This line updates the column index based on the specified change in the column direction (`dc`).

- `if row < 0 or row >= 6 or col < 0 or col >= 7 or self.board[row][col] != token:`: This line checks if the updated cell position is out of bounds or if the token in the cell is different from the token in the starting cell. If any of these conditions are met, it means that the sequence is broken, and the method returns `False`.

- `return True`: If the loop completes without breaking and all the tokens in the sequence match the token in the starting cell, the method returns `True`, indicating that there is a winning sequence in the specified direction.

The `check_line` method provides a flexible way to check for winning sequences in different directions on the game board, allowing the `check_winner` method to comprehensively determine if there is a winner in the game.


In [30]:
# Define the Connect Four game board
class ConnectFour:
    def __init__(self):
        self.board = [['.' for _ in range(7)] for _ in range(6)]  # Initialize the game board with empty spaces
        self.current_player = 'X'  # Set the starting player as X

    def display_board(self):
        for row in self.board:  # Iterate over each row in the board
            print(' '.join(row))  # Print the current row with spaces between tokens
        print()  # Print an empty line after displaying the board

    def drop_token(self, col):
        for row in range(5, -1, -1):  # Iterate from the bottom row to the top row
            if self.board[row][col] == '.':  # Check if the current cell is empty
                self.board[row][col] = self.current_player  # Place the current player's token in the cell
                return True  # Return True indicating a successful token drop
        return False  # Return False indicating the column is full and token cannot be dropped

    def switch_player(self):
        self.current_player = 'X' if self.current_player == 'O' else 'O'  # Switch the current player

    def check_winner(self):
        # Check for winning combinations horizontally, vertically, and diagonally
        for row in range(6):
            for col in range(7):
                if self.board[row][col] != '.':
                    if (self.check_line(row, col, 1, 0) or
                        self.check_line(row, col, 0, 1) or
                        self.check_line(row, col, 1, 1) or
                        self.check_line(row, col, 1, -1)):
                        return self.board[row][col]  # Return the winning player
        if all(cell != '.' for row in self.board for cell in row):
            return 'Draw'  # Return 'Draw' if the board is full and no player wins
        return None  # Return None if there is no winner yet

    def check_line(self, row, col, dr, dc):
        token = self.board[row][col]
        for _ in range(3):
            row += dr
            col += dc
            if row < 0 or row >= 6 or col < 0 or col >= 7 or self.board[row][col] != token:
                return False
        return True

## 3.3 Methods used for the evaluation

### 3.3.1 The `evaluate_segment` method

The `evaluate_segment` function is responsible for evaluating the score of a segment of four tokens in the Connect Four game. Here's how it works:

- `def evaluate_segment(segment):`: This line defines the function, which takes a single parameter `segment`. A segment represents a sequence of four adjacent cells in a row, column, or diagonal direction on the game board.

- `x_count = segment.count('X')` and `o_count = segment.count('O')`: These lines count the number of occurrences of 'X' and 'O' tokens in the segment, respectively.

- The function then evaluates the segment based on the counts of 'X' and 'O' tokens:
  - If there are four 'X' tokens in the segment (`x_count == 4`), it returns the constant `WIN_X`, indicating that the segment represents a winning sequence for player 'X'.
  - If there are four 'O' tokens in the segment (`o_count == 4`), it returns the constant `WIN_O`, indicating that the segment represents a winning sequence for player 'O'.
  - If there are no 'X' tokens in the segment (`x_count == 0`), it returns a score based on the count of 'O' tokens in the segment, using the `SEGMENT_VALUES` dictionary to look up the score.
  - If there are no 'O' tokens in the segment (`o_count == 0`), it returns a score based on the count of 'X' tokens in the segment, using the `SEGMENT_VALUES` dictionary to look up the score.
  - If the segment contains a mix of 'X' and 'O' tokens, it returns `0`, indicating that the segment does not contribute to a winning sequence.

The `evaluate_segment` function is used as part of the overall evaluation process in the Connect Four game to assess the strength of different segments of the game board and determine the overall score for a particular board configuration.

### 3.3.1 The `evaluate` method

The `evaluate` method is responsible for evaluating the overall score of the game board in the Connect Four game. Here's a breakdown of how it works:

Here's an explanation of each part of the `evaluate` method:

- **Initialization**: The `score` variable is initialized to zero, which will hold the cumulative score of the board evaluation.

- **Horizontal Evaluation**: The method iterates over each row of the board and extracts segments of four adjacent tokens horizontally. It then evaluates each segment using the `evaluate_segment` function and updates the `score` accordingly.

- **Vertical Evaluation**: Similarly, the method iterates over each column of the board and extracts segments of four adjacent tokens vertically. It evaluates each segment and updates the `score`.

- **Diagonal Evaluation**: The method evaluates segments diagonally, both from top-left to bottom-right and from bottom-left to top-right. It extracts each diagonal segment, evaluates it, and updates the `score`.

- **Move Bonus**: Finally, the method calculates a move bonus based on the difference in the counts of 'X' and 'O' tokens on the board. If the number of 'X' tokens is equal to the number of 'O' tokens, it adds `MOVE_BONUS_X` to the score; otherwise, it adds `MOVE_BONUS_O`.

- **Return**: The method returns the final evaluation score, which represents the overall strength of the current game board configuration.

In [31]:
# Evaluate a segment of four tokens
def evaluate_segment(segment):
    x_count = segment.count('X')
    o_count = segment.count('O')
    if x_count == 4:
        return WIN_X
    elif o_count == 4:
        return WIN_O
    elif x_count == 0:
        return SEGMENT_VALUES[-o_count]
    elif o_count == 0:
        return SEGMENT_VALUES[x_count]
    else:
        return 0  

# Evaluate the entire game board
def evaluate(board):
    score = 0  # Initialize the score to zero

    # Evaluate segments horizontally
    for row in board:
        for i in range(len(row) - 3):
            segment = row[i:i+4]  # Extract a segment of four tokens
            score += evaluate_segment(segment)  # Evaluate the segment and update the score

    # Evaluate segments vertically
    for col in range(len(board[0])):
        for i in range(len(board) - 3):
            segment = [board[j][col] for j in range(i, i+4)]  # Extract a segment of four tokens
            score += evaluate_segment(segment)  # Evaluate the segment and update the score

    # Evaluate segments diagonally (top-left to bottom-right)
    for i in range(len(board) - 3):
        for j in range(len(board[0]) - 3):
            segment = [board[i+k][j+k] for k in range(4)]  # Extract a segment of four tokens
            score += evaluate_segment(segment)  # Evaluate the segment and update the score

    # Evaluate segments diagonally (bottom-left to top-right)
    for i in range(3, len(board)):
        for j in range(len(board[0]) - 3):
            segment = [board[i-k][j+k] for k in range(4)]  # Extract a segment of four tokens
            score += evaluate_segment(segment)  # Evaluate the segment and update the score

    # Add move bonus
    num_x = sum(row.count('X') for row in board)  # Count the number of 'X' tokens on the board
    num_o = sum(row.count('O') for row in board)  # Count the number of 'O' tokens on the board
    score += MOVE_BONUS_X if num_x == num_o else MOVE_BONUS_O  # Add a bonus score based on the difference in token counts

    return score  # Return the final evaluation score

## 3.4 The A* method

The `astar_search` function implements the A* search algorithm to find the best move in the Connect Four game. Here's a breakdown of how it works:

- **Initialization**: The function initializes `best_score` to negative infinity and `best_move` to `None`. These variables will be updated as the function iterates through each possible move to find the best one.

- **Iteration**: The function iterates over each column of the game board. For each column, it creates a temporary game state (`temp_connect_four`) by copying the current board state.

- **Token Drop**: It attempts to drop a token in the current column of the temporary game state using the `drop_token` method. If a token can be dropped in that column (i.e., the column is not full), it proceeds to evaluate the resulting board state.

- **Evaluation**: The function evaluates the temporary board state using the `evaluate` function, which calculates a score representing the strength of the board configuration.

- **Update Best Move**: If the score of the temporary board state is better than the current best score, the `best_score` and `best_move` variables are updated accordingly to reflect the new best move found.

- **Return**: Finally, the function returns the column index of the best move found, which represents the column where the current player should drop their token to make the best move according to the A* search algorithm.

In [32]:
def astar_search(connect_four):
    best_score = float('-inf')  # Initialize the best score with negative infinity
    best_move = None  # Initialize the best move as None
    
    # Iterate over each column in the game board
    for col in range(7):
        temp_connect_four = ConnectFour()  # Create a temporary game state
        temp_connect_four.board = [row[:] for row in connect_four.board]  # Copy the current board state
        
        # Try dropping a token in the current column of the temporary game state
        if temp_connect_four.drop_token(col):
            # Evaluate the temporary board state
            score = evaluate(temp_connect_four.board)
            
            # Update the best score and best move if a better move is found
            if score > best_score:
                best_score = score
                best_move = col
    
    return best_move  # Return the column index of the best move


## 3.5 The Class Node

Sure, let's break down each method of the `Node` class:

### 3.5.1 The `__init__` method
   - This is the constructor method for the `Node` class.
   - It initializes a new node with the given `state` representing the game state, and an optional `parent` node.
   - `self.state`: Stores the game state associated with this node.
   - `self.parent`: Stores the parent node of this node. If no parent is provided, it defaults to `None`.
   - `self.children`: A list to store child nodes of this node. Initially empty.
   - `self.visits`: Tracks the number of times this node has been visited during simulations in the Monte Carlo Tree Search (MCTS) algorithm.
   - `self.wins`: Tracks the number of simulated wins from this node during MCTS simulations.

### 3.5.2 The `is_fully_expanded` method
   - This method checks if the node has fully expanded, meaning it has generated child nodes for all possible actions (in the context of the Connect Four game, all possible columns where a token can be dropped).
   - It returns `True` if the number of child nodes is equal to 7 (the total number of columns in Connect Four), indicating that all possible actions have been explored.

### 3.5.1 The `add_child` method
   - This method adds a new child node to the current node.
   - It takes a `child_state` parameter representing the game state associated with the new child node.
   - It creates a new `Node` object with the provided `child_state` and sets its parent to the current node.
   - The new child node is then appended to the list of children of the current node.
   - Finally, it returns the newly created child node.

### 3.5.1 The `is_terminal`
   - This method checks if the game state associated with the node is terminal, meaning the game has ended (either in a win, loss, or draw).
   - It calls the `check_winner` method of the game state (`self.state`) to determine if there is a winner.
   - If the `check_winner` method returns a value other than `None`, indicating a winner or a draw, the method returns `True`. Otherwise, it returns `False`.

### 3.5.1 The `uct_value`
   - This method calculates the Upper Confidence Bound for Trees (UCT) value of the node, which is used to select nodes during the selection phase of the MCTS algorithm.
   - It takes two parameters: `total_visits`, representing the total number of visits to the parent node, and `c`, representing the exploration parameter (default value is 1.0).
   - If the node has not been visited (`self.visits == 0`), it returns positive infinity to encourage exploration of unvisited nodes.
   - Otherwise, it calculates the UCT value using the formula: `exploitation + exploration`, where `exploitation` is the ratio of wins to visits for the node, and `exploration` is the exploration term based on the number of visits to the parent node.
   - The UCT value balances between exploiting nodes with high win rates and exploring nodes with few visits.


In [33]:
# Node class for Monte Carlo Tree Search
class Node:
    def __init__(self, state, parent=None):
        self.state = state  # Stores the game state associated with this node
        self.parent = parent  # Stores the parent node of this node. If no parent is provided, it defaults to None
        self.children = []  # A list to store child nodes of this node. Initially empty
        self.visits = 0  # Tracks the number of times this node has been visited during simulations in the Monte Carlo Tree Search (MCTS) algorithm
        self.wins = 0  # Tracks the number of simulated wins from this node during MCTS simulations

    def is_fully_expanded(self):
        return len(self.children) == 7  # Checks if the node has fully expanded, meaning it has generated child nodes for all possible actions

    def add_child(self, child_state):
        child_node = Node(child_state, parent=self)  # Adds a new child node to the current node
        self.children.append(child_node)  # Appends the newly created child node to the list of children
        return child_node  # Returns the newly created child node

    def is_terminal(self):
        return self.state.check_winner() is not None  # Checks if the game state associated with the node is terminal, meaning the game has ended

    def uct_value(self, total_visits, c=1.41):
        if self.visits == 0:
            return float('inf')  # Returns positive infinity if the node has not been visited to encourage exploration of unvisited nodes
        exploitation = self.wins / self.visits  # Calculates the exploitation term, the ratio of wins to visits for the node
        exploration = c * math.sqrt(math.log(total_visits) / self.visits)  # Calculates the exploration term based on the number of visits to the parent node
        return exploitation + exploration  # Returns the Upper Confidence Bound for Trees (UCT) value of the node, balancing between exploitation and exploration

## 3.6 MonteCarloTreeSearch's Methods

### 3.6.1 The `backpropagate` method

The `backpropagate` function is responsible for updating the visit counts and win counts of nodes in the tree during the backpropagation phase of the Monte Carlo Tree Search (MCTS) algorithm.
- It takes two parameters: `node`, which is the current node being updated, and `result`, which indicates the outcome of the simulation ('X' for a win by 'X', 'O' for a win by 'O', or 'Draw').
- Inside the while loop, the function traverses up the tree by moving to the parent node (`node = node.parent`) until it reaches the root node.
- At each node visited during the traversal, it increments the visit count (`node.visits += 1`) to reflect that the node has been visited once more during the simulation.
- Depending on the outcome of the simulation (`result`), it updates the win count of the node:
  - If `result` is 'X', indicating a win by player 'X', it decrements the win count (`node.wins -= 1`) because the perspective is from 'X'.
  - If `result` is 'O', indicating a win by player 'O', it increments the win count (`node.wins += 1`) because the perspective is from 'O'.

### 3.6.2 The `select` method

The `select` function is used in the selection phase of the Monte Carlo Tree Search (MCTS) algorithm.
- It takes a `node` as input, representing the current node in the tree from which the selection process starts.
- The function iteratively selects child nodes based on the UCT (Upper Confidence Bound for Trees) value until it reaches a leaf node or an unexpanded node.
- Inside the while loop, it checks if the current node is fully expanded and not a terminal state. If it is, the function selects the best child node based on the UCT value using the `best_child` function.
- If the current node is not fully expanded and not a terminal state, it calls the `expand` function to expand the node by adding a child node.
- Once the loop terminates, it returns the selected node, which is either a leaf node, an unexpanded node, or the best child node based on the UCT value.


### 3.6.3 The `expand` method

The `expand` function is a key component of the Monte Carlo Tree Search (MCTS) algorithm. It is responsible for expanding the search tree by adding a new child node to the current node, representing a possible game state resulting from a valid move.

Here's how the `expand` function works:

- It takes a node as input, representing the current state of the game.
- It selects a random column (representing a potential move) to drop a token into the game board. This randomness introduces exploration into the search process, allowing the algorithm to consider a variety of potential moves.
- It creates a new game state by copying the board configuration of the parent node's state. This ensures that modifications to the game board in the new state do not affect the original state.
- It switches the current player in the new game state to simulate the effect of making a move.
- It drops a token into the selected column of the new game state, representing the outcome of the selected move.
- It creates a new child node using the modified game state and adds it to the list of children of the parent node.
- It returns the newly created child node, representing the expanded portion of the search tree.

In summary, the `expand` function plays a crucial role in the MCTS algorithm by allowing the exploration of new game states and potential moves, thereby expanding the search space and improving the algorithm's ability to find promising solutions.

### 3.6.4 The `best_child` method

The `best_child` function is responsible for selecting the best child node of a given parent node based on the UCT (Upper Confidence Bound for Trees) value. In the context of the Monte Carlo Tree Search (MCTS) algorithm, each child node represents a possible move from the current game state.

Here's how the `best_child` function works:

- It takes the parent node as input.
- It calculates the total number of visits to all child nodes of the parent node.
- It iterates over each child node of the parent node and calculates the UCT value for each child node using the formula:

   \[ \text{UCT} = \frac{\text{wins}}{\text{visits}} + c \times \sqrt{\frac{\ln(\text{total\_visits})}{\text{visits}}} \]

   - `wins`: The number of wins obtained by the child node.
   - `visits`: The number of visits to the child node.
   - `total_visits`: The total number of visits to all child nodes of the parent node.
   - `c`: A constant that controls the balance between exploitation and exploration. Higher values of `c` encourage more exploration.

- It returns the child node with the highest UCT value, representing the most promising move to explore next in the MCTS algorithm.

In summary, the `best_child` function plays a crucial role in guiding the selection of child nodes during the exploration phase of MCTS, helping to prioritize moves that have shown promising outcomes in previous simulations.


### 3.6.5 The `simulate` method

The `simulate` function is a part of the Monte Carlo Tree Search (MCTS) algorithm, used to simulate the outcome of potential moves from a given game state. It plays a crucial role in the selection and expansion phases of MCTS by providing an estimate of the potential value of a game state through random simulation.

Here's how the `simulate` function works:

- Input: It takes a game state (represented by an instance of the `ConnectFour` class) as input.
- Initializatio: It creates a new instance of the `ConnectFour` class to simulate the game from the current state without modifying the original state.
- Token Dropping: It copies the board configuration from the input game state to the new instance, ensuring that any changes made during the simulation do not affect the original state.
- Current Player: It ensures that the current player in the new game state matches the current player in the input game state, simulating a continuation of the game.
- Move Evaluation: It iterates over each column in the game board and evaluates the potential value of dropping a token into that column. The evaluation is performed using the `evaluate` function, which assesses the quality of the resulting game state.
- Best Move Selection: It selects the column that yields the highest evaluated value as the simulated move.
- Result: It returns the index of the column representing the best move in the simulated game state.

In summary, the `simulate` function provides a mechanism for estimating the potential value of different moves from a given game state by simulating the outcome of each move and evaluating the resulting game state. This allows the MCTS algorithm to explore and prioritize promising moves during the search process.

### 3.6.6 The `mcts_search` method

The `mcts_search` function implements the Monte Carlo Tree Search (MCTS) algorithm, which is a decision-making algorithm commonly used in games and other domains with large decision spaces. MCTS operates by iteratively building and exploring a search tree based on random simulations to determine the best move to make from a given game state.

Here's a breakdown of how the `mcts_search` function works:

- Input: The function takes two parameters: `connect_four`, which represents the current game state (an instance of the `ConnectFour` class), and `num_simulations`, which specifies the number of simulations to perform during the search.

- Root Node Initialization: It initializes the root node of the search tree with the current game state.

- Simulation Phase: It iterates `num_simulations` times, performing the following steps in each iteration:
   - Selection: It selects a node in the search tree to explore based on the selection policy, typically using the Upper Confidence Bound for Trees (UCT) formula to balance exploration and exploitation.
   - Expansion: If the selected node has unexplored child nodes, it expands the search tree by adding a new child node corresponding to a potential game state resulting from a legal move.
   - Simulation: It simulates a complete game from the game state represented by the selected node. This simulation involves making random moves until reaching a terminal state (win, loss, or draw).
   - Backpropagation: It updates the statistics (number of visits and wins) of all nodes in the path from the selected node to the root based on the outcome of the simulation.

- Best Move Selection**: After the specified number of simulations, it selects the best move based on the statistics accumulated during the simulation phase. Typically, this involves choosing the move corresponding to the child node with the highest visit count.

- Output**: It returns the index of the column representing the best move to make from the current game state.

In summary, the `mcts_search` function employs the MCTS algorithm to explore the decision space of the game by simulating multiple possible sequences of moves, evaluating their outcomes, and ultimately selecting the move that maximizes the chances of achieving a favorable outcome.

In [34]:
# Backpropagate the simulation result through the tree
def backpropagate(node, result):
    while node:
        node.visits += 1  # Increment the visit count of the current node
        if result == 'X':
            node.wins -= 1  # Decrement the win count of the node if the result is a win for 'X'
        elif result == 'O':
            node.wins += 1  # Increment the win count of the node if the result is a win for 'O'
        node = node.parent  # Move to the parent node for further backpropagation

# Select the next node for exploration using UCT
def select(node):
    while node.is_fully_expanded() and not node.is_terminal():
        node = best_child(node)  # Select the best child node based on UCT value until a leaf or unexpanded node is reached
    if not node.is_fully_expanded() and not node.is_terminal():
        return expand(node)  # If the node is not fully expanded and not a terminal state, expand it
    return node  # Return the selected node

# Expand the tree by adding a new node
def expand(node):
    col = random.randint(0,6)
    new_state = ConnectFour()
    new_state.board = [row[:] for row in node.state.board]
    new_state.switch_player()
    new_state.drop_token(col)
    new_node = node.add_child(new_state)
    return new_node

# Choose the best child node based on UCT values
def best_child(node):
    total_visits = sum(child.visits for child in node.children)
    return max(node.children, key=lambda child: child.uct_value(total_visits))

# Simulate a game until completion and return the result
def simulate(connect_four):
    new_connect_four = ConnectFour()
    new_connect_four.board = [row[:] for row in connect_four.board]
    new_connect_four.current_player = connect_four.current_player
    
    best_move = None
    best_score = float('-inf')
    
    for col in range(7):
        if new_connect_four.drop_token(col):
            score = evaluate(new_connect_four.board)
            if score > best_score:
                best_score = score
                best_move = col
            new_connect_four.board = [row[:] for row in connect_four.board]
    
    return best_move

# Monte Carlo Tree Search algorithm
def mcts_search(connect_four, num_simulations=1000000):
    root = Node(connect_four)
    for _ in range(num_simulations):
        node_to_explore = select(root)
        simulated_move = simulate(node_to_explore.state)
        backpropagate(node_to_explore, simulated_move)
    return simulated_move

## 3.7 The Main method(s)

The `main` function serves as the entry point of the Connect Four game and orchestrates the interaction between the player and the AI using the selected algorithm. Here's a detailed explanation of how the `main` function works:

- Algorithm Selection: It prompts the user to choose between two algorithms: A* search and Monte Carlo Tree Search (MCTS). The user inputs either '1' for A* or '2' for Monte Carlo.

- Algorithm Assignment: Based on the user's choice, it assigns the appropriate search algorithm (`astar_search` for A* or `mcts_search` for Monte Carlo) to the `algorithm` variable.

- Connect Four Initialization: It creates an instance of the `ConnectFour` class, representing the game board and state.

- Game Loop: It enters a loop where the game continues until a winner is determined or the game ends in a draw. Within each iteration of the loop:
   - It displays the current state of the game board using the `display_board` method of the `ConnectFour` class.
   - It checks if there is a winner by invoking the `check_winner` method of the `ConnectFour` class. If a winner is found, it displays the result (win or draw) and exits the loop.
   - If it's the player's turn (current player is 'X'), it prompts the player to choose a column to make their move. The input is validated to ensure it's within the valid range (0 to 6) before proceeding.
   - If it's the AI's turn (current player is 'O'), it invokes the selected algorithm (`astar_search` or `mcts_search`) to determine the best move and updates the game state accordingly.
   - After each move, it switches the current player using the `switch_player` method of the `ConnectFour` class to alternate between 'X' and 'O'.

- Game Result Display: Once the game loop exits, indicating that the game has concluded, it displays the final state of the game board and the result (win or draw).

By organizing the game logic within the `main` function, it provides a clear structure for managing the game flow, player interactions, and AI decision-making, resulting in a cohesive and playable Connect Four game.

In [ ]:
# Main function to run the game
def main():
    # Choose the algorithm
    algorithm_choice = input("Choose the algorithm:\n1. A*\n2. Monte Carlo\nEnter the number of the algorithm you want to use: ")

    if algorithm_choice == '1':
        algorithm = astar_search
    elif algorithm_choice == '2':
        algorithm = mcts_search
    else:
        print("Invalid choice. Please enter '1' for A* or '2' for Monte Carlo.")
        return

    connect_four = ConnectFour()  # Initialize the game
    while True:
        connect_four.display_board()  # Display the current board
        winner = connect_four.check_winner()  # Check for a winner
        if winner:
            if winner == 'Draw':
                print("It's a draw!")  # Display draw message
            else:
                print(f"Player {winner} wins!")  # Display winner message
            break
        if connect_four.current_player == 'X':
            print("It's now X's turn.")  # Display X's turn message
            while True:
                col = int(input("Make a move by choosing your column (0-6): "))  # Ask for user input
                if 0 <= col <= 6:  # Check if the column is within the valid range
                    break
                else:
                    print("Invalid column number. Please enter a number between 0 and 6.")  # Display error message
            if connect_four.drop_token(col):  # Drop X's token in the column
                connect_four.switch_player()  # Switch to the next player
        else:
            print("Computer (O) is thinking...")  # Display O's turn message
            col = algorithm(connect_four)  # Find the best move using the selected algorithm
            if connect_four.drop_token(col):   # Drop O's token in the column
                connect_four.switch_player()   # Switch to the next player

if __name__ == "__main__":
    main()


# 4 Results
The results obtained from the solution using A* search and Monte Carlo Tree Search (MCTS) can vary based on several factors such as the complexity of the game, the heuristic used, and the number of simulations for MCTS. Here's a discussion on the results, insights, limitations, and potential improvements for each algorithm.

## 4.1 A* Search

   - **Results**: A* search tends to perform well in games like Connect Four where the state space is relatively small and the optimal move can be accurately determined with a heuristic evaluation function. It often finds the optimal move or a near-optimal move that leads to victory.
   - **Insights**: A* search explores the game tree using a heuristic evaluation function to estimate the desirability of each state. It guarantees finding the optimal solution when the heuristic is admissible and consistent.
   - **Limitations**: While A* search is effective in Connect Four, its performance heavily relies on the quality of the heuristic function. If the heuristic is not informative or overly optimistic, A* may explore unnecessary branches of the game tree, leading to suboptimal decisions.
   - **Potential Improvements**: Improving the heuristic function to better estimate the value of game states can enhance the performance of A* search. Additionally, optimizing the algorithm for efficiency, such as pruning redundant branches or using more sophisticated data structures, can speed up the search process.

## 4.2  Monte Carlo Tree Search (MCTS)

   - **Results**: MCTS has the potential to perform well in Connect Four by simulating multiple game trajectories and selecting moves based on their success rates in simulations. However, its performance may vary depending on factors such as the number of simulations and the exploration-exploitation balance.
   - **Insights**: MCTS is a sampling-based algorithm that iteratively builds a search tree by selecting nodes to explore and expanding them with simulated playouts. It balances exploration (visiting unexplored nodes) and exploitation (choosing moves with high success rates).
   - **Limitations**: MCTS may struggle in Connect Four if the search space is large, and the number of simulations is insufficient to adequately explore the game tree. It may also suffer from biased exploration if the selection strategy favors certain moves excessively.
   - **Potential Improvements**: To improve the performance of MCTS in Connect Four, increasing the number of simulations can lead to more accurate estimations of move values.

In summary, while A* search tends to work well in Connect Four due to its ability to find optimal solutions with a good heuristic, MCTS can be effective with appropriate tuning of parameters and strategies. Improving the heuristic evaluation function and enhancing the search strategy can lead to better performance for both algorithms in playing Connect Four.